# Panel 1

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 
            
            # Data
            library(dplyr), 
            
            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(ComplexHeatmap), 
            library(grid) # For gpar
            
        )
    )
)

In [ ]:
options(warn=-1)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO") # You need to adjust this path. I will help. 

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/dea_pl.R")
source("bin/gsea_pp.R")
source("bin/gsea_pl.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import and parameter 

In [ ]:
singler <- read.csv("result/singler/bulkRNAseq/singler_res.csv", row.names=1)

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_0.rds")
so <- AddMetaData(so, singler)

In [ ]:
so <- NormalizeData(so)

# SingleR UMAP

In [ ]:
fp_1 <- list(

    fplot(so, features="cmo_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="cmo_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4), 
    fplot(so, features="rpm_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="rpm_score", pt_size=0.25, min_cutoff=0, order=TRUE, restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4), 
    fplot(so, features="Spic", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="Spic", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4), 
    fplot(so, features="Gdf15", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_NaCl_D6"), size_select=4), 
    fplot(so, features="Gdf15", pt_size=0.25, min_cutoff=0, order=TRUE, slot="data", assay="RNA", restrict=c("sample_group", "Bl6_CpG_D6"), size_select=4)

    
)

In [ ]:
fp_1 <- lapply(fp_1, function(p) egg::set_panel_size(p, width=unit(2.5, "cm"), height=unit(2.5, "cm")))
fp_1 <- do.call(gridExtra::arrangeGrob, c(fp_1, ncol=2, nrow=length(fp_1)/2))

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/fp_rpm_score.pdf", width=2*2.0, height=1.5*length(fp_1)/2)

grid::grid.draw(fp_1)

dev.off()

# DEA result count heatmap

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")
res_dea_3 <- readRDS("result/dea/bulkRNAseq/res_dea_ery.rds")

In [ ]:
genotype_wald_1 <- purrr::reduce(
    
    list(
        
        res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[1]] %>% dplyr::mutate(Ctl=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
        res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% dplyr::mutate(D1=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
        res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[3]] %>% dplyr::mutate(D3=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
        res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]][[4]] %>% dplyr::mutate(D6=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
    
    ), full_join, by="gene"

) %>% dplyr::select(-gene) %>% dplyr::distinct()

In [ ]:
genotype_wald_2 <- purrr::reduce(
    
    list(
        
        res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[1]] %>% dplyr::mutate(Ctl=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
        res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% dplyr::mutate(D1=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
        res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[3]] %>% dplyr::mutate(D3=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
        res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]][[4]] %>% dplyr::mutate(D6=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
    
    ), full_join, by="gene"

) %>% dplyr::select(-gene) %>% dplyr::distinct()

In [ ]:
genotype_wald_3 <- purrr::reduce(
    
    list(
        
        res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[1]] %>% dplyr::mutate(Ctl=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
        res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[2]] %>% dplyr::mutate(D1=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
        res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[3]] %>% dplyr::mutate(D3=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
        res_dea_3[["res"]][["dpi_wt_vs_ko_wald"]][[4]] %>% dplyr::mutate(D6=sum((ihw<=0.05 & abs(log2FoldChange)>=0.25))) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
    
    ), full_join, by="gene"

) %>% dplyr::select(-gene) %>% dplyr::distinct()

In [ ]:
mat_1 <- rbind(genotype_wald_1, genotype_wald_2, genotype_wald_3)
rownames(mat_1) <- c("cMo", "RPM", "EB")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_count.pdf", width=1.5, height=3)

draw(dea_count_hm(mat_1, fontsize_select=2))

dev.off()

# Gene set enrichment analysis (GSEA)

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.25

## Prepare results from genotype wald test

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")

In [ ]:
res_dea <- list("cMo"=res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], "RPM"=res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]])

## genotype comparison - HM

In [ ]:
gsea_res_1 <- lapply(seq_along(res_dea[[1]]), function(i) gsea(res_dea[[1]][[i]] %>% dplyr::rename(avg_log2FC=log2FoldChange, p_val_adj=ihw), category="MH", subcategory=NULL) %>% dplyr::mutate(col_split="cMo", col_label=paste0("group_", i)))
gsea_res_2 <- lapply(seq_along(res_dea[[2]]), function(i) gsea(res_dea[[2]][[i]] %>% dplyr::rename(avg_log2FC=log2FoldChange, p_val_adj=ihw), category="MH", subcategory=NULL) %>% dplyr::mutate(col_split="RPM", col_label=paste0("group_", i)))

In [ ]:
gsea_res_1 <- do.call("rbind", gsea_res_1)
gsea_res_2 <- do.call("rbind", gsea_res_2)

In [ ]:
gsea_res <- rbind(gsea_res_1, gsea_res_2)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=0) %>% dplyr::pull(pathway) %>% unique()

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
mat_1 <- mat_1[, c("col_label", "col_split", gs_name_select)]
mat_2 <- mat_2[, c("col_label", "col_split", gs_name_select)]

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_gsea_genotype_dpi_mh.pdf", width=8, height=10)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="HALLMARK", color_neg=color$sample_group[1], color_pos=color$sample_group[2], fontsize_select=2)

dev.off()

# DEA heatmap 

In [ ]:
adj_pval_thr <- 0.05
log2FC_thr <- 0.25

In [ ]:
hm_dea_fetch <- function(mat_1, dea_res, col_samples, col_split, fontsize_select, celltype, celltype_select, scale_mode="all") {

    mat_1 <- Seurat::CreateSeuratObject(counts=mat_1)
    mat_1 <- NormalizeData(mat_1, normalization.method="RC", scale.factor=1e6)
    mat_1 <- GetAssayData(mat_1, assay="RNA", layer="data") %>% as.data.frame() 

    if(scale_mode=="all") {

        mat_1 <- t(apply(mat_1, 1, scales::rescale))
        mat_1 <- mat_1[row_split, col_samples]

        colnames(mat_1) <- col_label

        row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
        row_split <- row_split[row_split %in% rownames(mat_1)]
        
        mat_1 <- mat_1[row_split, celltype==celltype_select]
        col_split <- col_split[celltype==celltype_select]
        
    } else if (scale_mode=="sample") {

        mat_1 <- mat_1[row_split, col_samples]
        mat_1 <- mat_1[row_split, col_samples]

        colnames(mat_1) <- col_label

        row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
        row_split <- row_split[row_split %in% rownames(mat_1)]
        
        mat_1 <- mat_1[row_split, celltype==celltype_select]
        col_split <- col_split[celltype==celltype_select]
        
        mat_1 <- t(apply(mat_1, 1, scales::rescale))
        
    }
    
    # DEA log2FC 
    mat_2 <- purrr::reduce(
        
        list(
            
            res_dea[[celltype_select]][[1]] %>% dplyr::mutate(Ctl=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
            res_dea[[celltype_select]][[2]] %>% dplyr::mutate(D1=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
            res_dea[[celltype_select]][[3]] %>% dplyr::mutate(D3=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
            res_dea[[celltype_select]][[4]] %>% dplyr::mutate(D6=log2FoldChange) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_2[setdiff(row_split, rownames(mat_2)), ] <- 0
    mat_2 <- mat_2[row_split, ]
    mat_2[is.na(mat_2)] <- 0

    # DEA adj p-values
    mat_3 <- purrr::reduce(
        
        list(
            
            res_dea[[celltype_select]][[1]] %>% dplyr::mutate(Ctl=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, Ctl), 
            res_dea[[celltype_select]][[2]] %>% dplyr::mutate(D1=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D1), 
            res_dea[[celltype_select]][[3]] %>% dplyr::mutate(D3=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D3), 
            res_dea[[celltype_select]][[4]] %>% dplyr::mutate(D6=padj) %>% tibble::rownames_to_column("gene") %>% dplyr::select(gene, D6)
        
        ), full_join, by="gene"
    
    ) %>% tibble::column_to_rownames("gene")

    mat_3[setdiff(row_split, rownames(mat_3)), ] <- 1
    mat_3 <- mat_3[row_split, ]
    mat_3[is.na(mat_3)] <- 1
    mat_3[(mat_3>adj_pval_thr & abs(mat_2)<log2FC_thr)] <- 1

    # Set log2FC to 0 if not significant 
    mat_2[abs(mat_2) < log2FC_thr] <- 0

    # Row split
    row_split=names(row_split)
    row_split=factor(row_split, levels=names(row_genes))
    
    hm_1 <- dea_exp_hm(mat_1, col_split=col_split, row_split=row_split, fontsize_select=2, breaks_limit=2, scale_width=0.5, legend_title="Exp.", cluster_rows=FALSE)
    hm_2 <- dea_res_hm(mat_2, mat_3, p_val_adj_thr=adj_pval_thr, row_split=row_split, scale_width=1, color_min="#cd34b5", color_max="#FFAC1E", cluster_rows=FALSE, fontsize_select=2)

    hm <- hm_2 + hm_1

    return(hm)
    
}

# Prepare meta data

In [ ]:
meta <- readRDS("data/bulkRNAseq/object/meta.rds") %>% dplyr::filter(celltype %in% c("cMo", "RPM"))

In [ ]:
meta <- meta %>% dplyr::arrange(celltype, dpi, genotype)

col_samples <- as.character(meta$sample_name)
col_label <- as.character(meta$genotype)
col_split <- as.character(meta$dpi)
celltype <- as.character(meta$celltype)

## Prepare results from genotype wald test

In [ ]:
res_dea_1 <- readRDS("result/dea/bulkRNAseq/res_dea_cmo.rds")
res_dea_2 <- readRDS("result/dea/bulkRNAseq/res_dea_rpm.rds")

In [ ]:
res_dea_1[["resc"]][["pc_wt_vs_ko"]] <- res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::mutate(response_class_wt=ifelse(response_gene_wt, response_class_wt, NA), response_class_ko=ifelse(response_gene_ko, response_class_ko, NA))
res_dea_2[["resc"]][["pc_wt_vs_ko"]] <- res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::mutate(response_class_wt=ifelse(response_gene_wt, response_class_wt, NA), response_class_ko=ifelse(response_gene_ko, response_class_ko, NA))

In [ ]:
res_dea_1[["resc"]][["pc_wt_vs_ko"]] <- res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter((response_gene_wt | response_gene_ko) & perturbation_gene) %>% dplyr::filter(!(is.na(response_class_wt) & is.na(response_class_ko)))
res_dea_2[["resc"]][["pc_wt_vs_ko"]] <- res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter((response_gene_wt | response_gene_ko) & perturbation_gene) %>% dplyr::filter(!(is.na(response_class_wt) & is.na(response_class_ko)))

In [ ]:
res_dea <- list("cMo"=res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], "RPM"=res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]])

## Prepare z-score mat

In [ ]:
meta <- readRDS("data/bulkRNAseq/object/meta.rds") %>% dplyr::filter(celltype %in% c("cMo", "RPM")) %>% droplevels()

In [ ]:
mat_1 <- readRDS("data/bulkRNAseq/object/cnt.rds") %>% dplyr::select(-mgi_symbol, -ensembl_gene_id)
mat_1 <- mat_1[, rownames(meta)]

In [ ]:
all(rownames(meta)==colnames(mat_1))

## cMo cre/+ UP vs +/+

In [ ]:
gene_select <- lapply(res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange>=log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="cMo", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_up_vs_wt_cmo.pdf", width=10, height=70)

hm_1

dev.off()

## cMo cre/+ DOWN vs +/+

In [ ]:
gene_select <- lapply(res_dea_1[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange<=-log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_1[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="cMo", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_down_vs_wt_cmo.pdf", width=10, height=80)

hm_1

dev.off()

## RPM cre/+ UP vs +/+

In [ ]:
gene_select <- lapply(res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange>=log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="RPM", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_up_vs_wt_rpm.pdf", width=10, height=30)

hm_1

dev.off()

## RPM cre/+ DOWN vs +/+

In [ ]:
gene_select <- lapply(res_dea_2[["res"]][["dpi_wt_vs_ko_wald"]], function(x) x %>% dplyr::filter(ihw<=adj_pval_thr & log2FoldChange<=-log2FC_thr) %>% rownames())
gene_select <- do.call("c", gene_select) %>% unique()
gene_select <- gene_select[!gene_select %in% grep("Gm|Rps|Rpl", gene_select, value=TRUE)]

In [ ]:
row_genes <- list(

    "Repressed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & response_class_ko=="Repressed") %>% rownames(), 
    "Activated"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & response_class_ko=="Activated") %>% rownames(), 
    "Activated (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Activated" & is.na(response_class_ko)) %>% rownames(), 
    "Activated (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Activated") %>% rownames(), 
    "Repressed (+/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt=="Repressed" & is.na(response_class_ko)) %>% rownames(), 
    "Repressed (cre/+)"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(is.na(response_class_wt) & response_class_ko=="Repressed") %>% rownames(),
    "Opposed"=res_dea_2[["resc"]][["pc_wt_vs_ko"]] %>% dplyr::filter(response_class_wt!=response_class_ko) %>% rownames()
    
)

In [ ]:
row_genes <- lapply(row_genes, function(x) x[x %in% gene_select])

In [ ]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))

In [ ]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="RPM", scale_mode="sample")

In [ ]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_ko_down_vs_wt_rpm.pdf", width=10, height=25)

hm_1

dev.off()

# Selected genes

In [638]:
row_genes <- list(

    "G1"=c("Ccr2", "Ly6c2", "Spic", "Cd68", "Cd163"), 
    "G2"=c("Vcam1", "Itga9"), 
    "G3"=c("Ccl24", "Kitl", "Gdf15", "Il1b", "Il18bp", "Il10", "Ccl2", "Ccl12", "Ccl4", "Cxcl10"), 
    "G4"=c("Slc40a1", "Slc48a1", "Ftl1", "Fth1", "Hmox1", "Hmox2", "Nfe2l2", "Acsl4", "Sod2", "Prdx1"), 
    "G5"=c("Stat1", "Stat2", "Irf9", "Irf7", "Ifih1", "Isg15", "Isg20", "Ifi44", "Ifit1", "Ifit2", "Ifit3", "Ifitm1", "Ifitm3", "Usp18", "Mx1", "Oasl1", "Oasl2", "Ddx60", "Cd274")
    
)

In [639]:
row_split <- setNames(unlist(row_genes, use.names=FALSE), rep(names(row_genes), sapply(row_genes, length)))
row_split[!row_split %in% rownames(mat_1)]

named character(0)

In [640]:
hm_1 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="cMo", scale_mode="all")
hm_2 <- hm_dea_fetch(mat_1, dea_res, col_samples, col_split, fontsize_select=2, celltype, celltype_select="RPM", scale_mode="all")

Normalizing layer: counts

Normalizing layer: counts



In [641]:
pdf("result/figures/2_bulkRNAseq/panel_1/hm_dea_cmo_rpm_selected.pdf", width=10, height=10)

hm_1 + hm_2

dev.off()

pdf 
  2

# Session Info

In [ ]:
sessionInfo()